In [9]:
from pyspark.sql import *
import pyspark
from pyspark.sql import functions as f
from pyspark.sql.window import Window

import pandas as pd

In [101]:
data_pandas = pd.DataFrame(\
    {'employee_number':['801','1003','1019','1016','1008','1006','1014','1011','1010','1001','1004','1012','1002','1015'],\
     'manager_employee_number':[None,'801','801','801','1019','1019','1019','1019','1003','1003','1003','1004','1004','1004']})
data_pandas.employee_number = data_pandas.employee_number.astype(str)
data_pandas.manager_employee_number = data_pandas.manager_employee_number.astype(str)
display(data_pandas)

,employee_number,manager_employee_number
0,801,None
1,1003,801
2,1019,801
3,1016,801
4,1008,1019
5,1006,1019
6,1014,1019
7,1011,1019
8,1010,1003
9,1001,1003


In [103]:
df_employee_rec = spark.createDataFrame(data_pandas).coalesce(1)
df_employee_rec.show(20)

+---------------+-----------------------+
|employee_number|manager_employee_number|
+---------------+-----------------------+
|            801|                   None|
|           1003|                    801|
|           1019|                    801|
|           1016|                    801|
|           1008|                   1019|
|           1006|                   1019|
|           1014|                   1019|
|           1011|                   1019|
|           1010|                   1003|
|           1001|                   1003|
|           1004|                   1003|
|           1012|                   1004|
|           1002|                   1004|
|           1015|                   1004|
+---------------+-----------------------+



In [104]:
df_employee_rec.registerTempTable("employee_rec")
df_employee_seed = spark.sql("select employee_number from employee_rec where manager_employee_number = 'None'")
df_employee_seed.show()

df_cnt = 1
cnt = 1
df_employee_seed.registerTempTable("vt_seed0")
print(f"df_cnt:{df_cnt}")
print(f"cnt:{cnt}")

+---------------+
|employee_number|
+---------------+
|            801|
+---------------+

df_cnt:1
cnt:1


In [105]:
while (df_cnt != 0):
    print(f"df_cnt:{df_cnt}")
    tblnm = "vt_seed" + str(cnt-1) ;
    print(f"tblnm:{tblnm}")
    tblnm1 = "vt_seed" + str(cnt) ;
    print(f"tblnm1:{tblnm1}")
    df_employee_rec = spark.sql(f"select indirect.employee_number FROM {tblnm} direct, employee_rec indirect WHERE direct.employee_number = indirect.manager_employee_number");
    df_cnt=int(df_employee_rec.count());
    if(df_cnt!=0):
        print(f'register temp table:{tblnm1}')
        df_employee_rec.registerTempTable(tblnm1);

    cnt = cnt + 1;

df_cnt:1
tblnm:vt_seed0
tblnm1:vt_seed1
register temp table:vt_seed1
df_cnt:3
tblnm:vt_seed1
tblnm1:vt_seed2
register temp table:vt_seed2
df_cnt:7
tblnm:vt_seed2
tblnm1:vt_seed3
register temp table:vt_seed3
df_cnt:3
tblnm:vt_seed3
tblnm1:vt_seed4
+---------------+
|employee_number|
+---------------+
+---------------+



In [109]:
print("empty sons:")
spark.sql("select * from vt_seed0").show()
print("801 sons:")
spark.sql("select * from vt_seed1").show()
print("1003|1019|1016 sons:")
spark.sql("select * from vt_seed2").show()
print("1010|1001|1004|1008|1006|1014|1011 sons:")
spark.sql("select * from vt_seed3").show()

empty sons:
+---------------+
|employee_number|
+---------------+
|            801|
+---------------+

801 sons:
+---------------+
|employee_number|
+---------------+
|           1003|
|           1019|
|           1016|
+---------------+

1003|1019|1016 sons:
+---------------+
|employee_number|
+---------------+
|           1010|
|           1001|
|           1004|
|           1008|
|           1006|
|           1014|
|           1011|
+---------------+

1010|1001|1004|1008|1006|1014|1011 sons:
+---------------+
|employee_number|
+---------------+
|           1012|
|           1002|
|           1015|
+---------------+



In [111]:
fin_query = ""
values = [(a) for a in range(0,(cnt-1))]
for value in values:
    if (value == 0):
        fin_query = fin_query + "select employee_number from vt_seed" + str(value)
    else:
        fin_query = fin_query+ " union select employee_number from vt_seed" + str(value)
        
print(f"final query:\n{fin_query}")

final query:
select employee_number from vt_seed0 union select employee_number from vt_seed1 union select employee_number from vt_seed2 union select employee_number from vt_seed3


In [112]:
final_result_df = spark.sql(fin_query)
print("full list of descendant:")
final_result_df.sort(f.asc("employee_number") ).show()

full list of descendant:
+---------------+
|employee_number|
+---------------+
|           1001|
|           1002|
|           1003|
|           1004|
|           1006|
|           1008|
|           1010|
|           1011|
|           1012|
|           1014|
|           1015|
|           1016|
|           1019|
|            801|
+---------------+

